<a href="https://colab.research.google.com/github/HaitaanshDixit/Research_Paper/blob/main/Fed_Learning_flw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flwr==1.9.0
!pip install transformers torch torchvision accelerate
!pip install ray[default]

In [ ]:
import flwr as fl
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
print(fl.__version__)

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#MODEL_NAME = "google/gemma-2-2b-it"
#tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
def load_model():
    return AutoModelForCausalLM.from_pretrained(
        "google/gemma-2-2b-it",
        device_map="auto",
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    )


In [ ]:
# Loading the datasets
from torch.utils.data import DataLoader

def load_dataset(language):
    dummy_data = {
        "hindi": [
            ("पूर्व केंद्रीय मंत्री जयराम रमेश ने सोमवार को एक प्रेस वार्ता में कहा, भगोड़े ललित मोदी और वसुंधरा राजे में लेन-देन का रिश्ता है. भगोड़े ललित मोदी देते हैं और वसुंधरा लेती हैं. उन्होंने दस्तावेज़ पेश करते हुए आरोप लगाया कि ललित मोदी और वसुंधरा राजे ने मिलकर राजस्थान सरकार की संपत्ति को निजी संपत्ति बनाया है. उन्होंने कहा, 1954 से लेकर आज तक के दस्तावेज़ों में धौलपुर महल सरकारी संपत्ति है लेकिन ललित मोदी और वसुंधरा परिवार की कंपनी इसका निजी इस्तेमाल कर रही है. जयराम रमेश ने कहा कि वसुंधरा राजे के ख़िलाफ़ ठोस दस्तावेज़ हैं और अब प्रधानमंत्री को उन पर कार्रवाई करनी ही चाहिए. उन्होंने ये भी कहा कि सवाल यह नहीं है कि वसुंधरा को जाना होगा या नहीं सवाल यह है कि वो कब और कैसे अलविदा होंगी, उन्होंने कहा मौनेंद्र अगर धृतराष्ट्र बनना चाहते हैं तो उनकी मर्ज़ी है. उन्होंने आगे कहा, नैतिकता की वकालत करने वाले मौनेंद्र जी को अपनी नींद से उठना होगा और जाँच की ज़रूरत नहीं है, जो दस्तावेज़ मौजूद हैं उन्हें के आधार पर उनका इस्तीफ़ा लेना होगा. आज आम आदमी पार्टी भी दिल्ली में ललित मोदी स्कैंडल में फँसे बीजेपी नेताओं के ख़िलाफ़ प्रदर्शन कर रही है. ट्विटर पर भी 'दागी मंत्री मोदी के' के ट्रेंड कर रहा है.",
             "आईपीएल के पूर्व कमिश्नर और वित्तीय अनियमितताओं की जाँच में फँसे ललित मोदी की मदद के आरोपों में घिरी राजस्थान की मुख्यमंत्री वसुंधरा राजे पर कांग्रेस ने ताज़ा आरोप लगाए हैं.")
            ,
            ("यह कहना है अभिनेत्री ऋचा चड्ढा का जिनकी फ़िल्म 'तमंचे' इस शुक्रवार को रिलीज़ हुई. बीबीसी से ख़ास बातचीत में ऋचा ने कहा, मेरे नाम के आगे ख़ान या चोपड़ा लगा होता तो शायद मेरे लिए राह आसान होती. तो क्या ऋचा को अफ़सोस है? जवाब में वह बोलीं, मैं बड़े नामी-गिरामी निर्माताओं के साथ काम करना मिस नहीं करती क्योंकि मुझे मेरे काम से प्यार है. मैं काम पैसों के लिए नहीं अपनी ख़ुशी के लिए करती हूं. लेकिन ऋचा को बॉलीवुड की स्टार कल्चर से शिक़ायत है. वह कहती हैं, बड़े बैनर तो सिर्फ़ स्टार के साथ ही काम करते हैं. छोटे निर्देशकों की फ़िल्में भी जब हिट हो जाती हैं, तो वो अपनी अगली फ़िल्म किसी बड़े सितारे के साथ करना चाहते हैं. ऋचा अभय देओल के साथ फ़िल्म 'ओए लकी लकी ओए' में पहली बार दिखी थीं लेकिन इसे वह अपना ब्रेक नहीं मानतीं. वो कहती हैं, इस फ़िल्म के बाद भी डेढ़ साल तक मैं खाली थी. मैं अपना असल ब्रेक मानती हूं गैंग्स ऑफ़ वासेपुर को.'गैंग्स ऑफ़ वासेपुर' में मनोज बाजपेयी की पत्नी के रूप में ऋचा के अभिनय को वाहवाही मिली थी. इसके अलावा संजय लीला भंसाली के साथ काम करने की चाहत में उन्होंने 'गोलियों की रासलीला राम लीला' में एक छोटा सा रोल भी क़बूल कर लिया. 'तमंचे' में उनके साथ निखिल द्विवेदी हैं. ऋचा ने एक चोर का किरदार निभाया है.",
             "अगर आप नए हैं और आपके पास एक मशहूर ख़ानदान का नाम नहीं है, तो मुश्किलें ज़रूर आएंगी.")
        ],
        "marathi": [
            ("या विमानात 62 प्रवासी होते. अधिकाऱ्यांनी दिलेल्या माहितीनुसार, श्रीविजया एयर बोइंग 737चा जकार्ताच्या कलिमनतन प्रांतातल्या रस्त्यात संपर्क तुटला. त्यानंतर विमान बेपत्ता झालं. फ्लाइट ट्रैकिंग वेबसाइट फ्लाइटरडार24.कॉम नुसार, हे विमान एका मिनिटापेक्षा कमी वेळात 10 हजार फूट खाली आलं. परिवहन मंत्रालयानं सांगितलंय की, विमानाचा शोध घेण्यासाठी बचावपथकांना सक्रिय करण्यात आलं आहे.  या उड्डाणासंबंधी अधिक माहिती घेत असल्याचं श्रीविजया एअरनं म्हटलं आहे. बेपत्ता झालेल्या विमानाशी शेवटचा संपर्क स्थानिक वेळेनुसार दुपारी 2 वाजून 40 मिनिटांनी झाला असल्याची माहिती अधिकाऱ्यांनी दिली.  जकार्तामधील बीबीसी प्रतिनिधी जेरोमी विरावान यांच म्हणणं आहे की, विमान अपघाताची ही घटना इंडोनेशियासमोर काही प्रश्नं उपस्थित करणारी आहे. या देशाची एअरलाइन इंडस्ट्री 2018 मधील भीषण विमान अपघातानंतर अतिशय कठोर अशा समीक्षेला सामोरी जात आहे.  नॅशनल सर्च अँड रेस्क्यू एजन्सीचे एक अधिकारी बामबैंग सुरयो अजी यांनी म्हटलं, आम्ही अपघाताचं अचूक स्थान शोधण्याचा प्रयत्न करत आहोत. आज रात्रीपर्यंत त्याचा पत्ता लागेल, अशी आम्हाला आशा आहे. समुद्राची खोली जवळपास 20-23 मीटर आहे. एका प्रत्यक्षदर्शीनं एका मोठा आवाज ऐकल्याचं सांगितलं. सोलिहिन नावाच्या एका मच्छीमारानं बीबीसीच्या इंडोनेशियन सर्व्हिसशी बोलताना म्हटलं की, त्यांनी हा अपघात होताना पाहिला. त्यानंतर त्यांनी त्यांच्या कॅप्टनसोबत बेटावर परतण्याचा निर्णय घेतला. त्यांनी म्हटलं, विमान वीजेच्या वेगानं समुद्रात कोसळलं आणि मोठ्ठा आवाज झाला. आम्ही जवळच होतो. काही तुकडे आमच्या जहाजावरही येऊन आदळले. बीबीसीचे प्रतिनिधी थियो लेगेट यांच्या मते इंडोनेशियातील अनेक विमानं खूप जुनी झाली आहेत.  शनिवारी बेपत्ता झालेलं विमान 26 वर्षं जुनं होतं. सुरक्षित विमान उड्डाणांबाबत इंडोनेशियाचा इतिहास फारसा चांगला नाहीये. ऐंशी आणि नव्वदच्या दशकात बनलेली अनेक विमानं इंडोनेशियात अजूनही वापरली जातात.  इंडोनेशियामध्ये याआधीही दोन मोठे विमान अपघात झाले होते, ज्यामध्ये 737 मॅक्स बोइंग विमानं दुर्घटनाग्रस्त झाली होती.  शनिवारी (9 जानेवारी) जकार्ताहून टेक ऑफ केलेलं विमान 737 मॅक्स श्रेणीतलं नाहीये.  ऑक्टोबर 2018 मध्ये इंडोनेशियातील लायन एअरच्या विमानाला अपघात झाला होता. या अपघातात 189 लोक मृत्यूमुखी पडले होते. ",
             "इंडोनेशियाची राजधानी जकार्ता इथून उड्डाणानंतर एक विमान बेपत्ता झालं आहे.")
            ,
            ("गेल्यावर्षीही कोरोनाच्या जागतिक साथीमुळे राज्यामध्ये गणेशोत्सवासाठीच्या विशेष सूचना देण्यात आल्या होत्या. यावर्षीसाठीही अशाच सूचना जाहीर करण्यात आल्या आहेत. १० सप्टेंबर (गणेश चतुर्थी) ते १९ सप्टेंबर (अनंत चतुर्दशी) या काळात राज्यात सार्वजनिक आणि घरगुती गणेशोत्सव साजरा केला जाईल. सार्वजनिक गणेशोत्सवासाठी मंडळांना महापालिका वा स्थानिक प्रशासनाची पूर्व परवानगी घ्यावी लागेल. राज्य सरकारने जाहीर केलेल्या मार्गदर्शक सूचना एकतर्फी असल्याचं सार्वजनिक गणेशोत्सव समन्वय समितीचं म्हणणं आहे. मुख्यमंत्री उद्धव ठाकरे यांनी गणेश मंडळांशी चर्चा करावी अशी मागणी समन्वय समितीचे प्रमुख नरेंद्र दहिबावकर यांनी केली आहे.ते म्हणाले, गेल्या दोन महिन्यांपासून सार्वजनिक गणेशोत्सव समन्वय समिती नियमावली ठरवण्यासाठी पत्रव्यवहार करत आहे. परंतु सरकारने आज परस्पर परिपत्रक जारी केलं. मंडळाच्या कार्यकर्त्यांचे लसीकरण आणि ऑनलाईन दर्शन यानुसार उत्सव साजरा करण्यासंदर्भात आम्ही चर्चेची मागणी करत होतो. आता सरकारने एकतर्फी निर्णय घेतलेला आहे. यामुळे सार्वजनिक मंडळांमध्ये नाराजी आहे. या निर्णयाचा फेरविचार करावा अशी मागणी आम्ही करत आहोत. यूट्यूब (बीबीसी न्यूज मराठीचे सर्व अद्ययावत माहिती मिळवण्यासाठी आम्हाला अनुसरावे)बीबीसी न्यूज मराठीच्या सगळ्या बातम्या तुम्ही जिओ टीव्ही अ‍ॅप वर पाहू शकता. 'सोपी गोष्ट' आणि '३ गोष्टी' हे मराठीतले बातम्यांचे पहिले ध्वनिचित्रफीती (पॉडकास्ट्स) तुम्ही गाना, स्पॉटिफाय, जिओसावन आणि अ‍ॅपल ध्वनिचित्रफीती येथे ऐकू शकता.",
             "राज्यातली कोरोना व्हायरसच्या संसर्गाची परिस्थिती लक्षात घेत राज्य सरकारने गणेशोत्सवासाठीची मार्गदर्शक तत्त्वं जाहीर केली आहेत. गणेशोत्सव साधेपणाने साजरा करण्याची सूचना करण्यात आली आहे.")
        ],
        "bengali": [
            ("কোতোয়ালি মডেল থানার পুলিশ কামরুল ইসলাম বিবিসিকে জানিয়েছেন, আহত হয়ে অনেকেই এখন হাসপাতালে রয়েছেন এবং এ ঘটনায় মৃতের সংখ্যা বাড়তে পারে। মিস্টার ইসলাম জানান, সকালে এলাকার ব্যবসায়ী ও নুরানি জর্দা ফ্যাক্টরির মালিক শামীম তালুকদারের বাড়িতে এ ঘটনা ঘটে। আজ ওই বাড়িতে যাকাতের কাপড় দেওয়ার কথা ছিল। এতে বহু আজ ভোরে সেখানে ভীড় করলে হুড়োহুড়ি শুরু হয় ও পদদলিত হওয়ার ঘটনা ঘটে। যাকাতদাতা ব্যবসায়ী শামীম তালুকদার সহ ৮ জনকে গ্রেপ্তার করেছে পুলিশ।",
             "বাংলাদেশের ঢাকার কাছে ময়মনসিংহ শহরে যাকাতের কাপড় সংগ্রহ করতে গিয়ে হুড়োহুড়িতে পদদলিত হয়ে কমপক্ষে ১৭ জন মারা গেছে ও আহত হয়েছে আরও অনেকে।")
            ,
            ("পেনিসিলভানিয়ার ওই দম্পতির ভাষ্য অনুযায়ী, পরীক্ষা-নিরীক্ষার সময় তাঁরা দেখতে পান বাম দিকে পোশাক পরিহিত এক ব্যক্তি তাদের শিশুর দিকে তাকিয়ে আছে। তারা যখন আল্ট্রাসাউন্ডের ছবিসহ রিপোর্ট আমাদের কাছে দিলো, আমার কাছে মনে হয়েছে ওই ব্যক্তি যীশুখ্রিস্ট-স্থানীয় একটি পত্রিকাকে বলেছেন গর্ভবতী আলিশিয়া জিক। তিনি আরো বলেছেন, এই রিপোর্ট পেয়ে তাঁরা স্বামী-স্ত্রী দুজনেই বেশ আশ্বস্ত হয়েছেন এবং প্রশান্তির মধ্যে আছেন। এর আগে আরো দুবার গর্ভধারণ করলেও, গর্ভধারণ সংক্রান্ত জটিলতায় দুটো সন্তানই সুস্থ হয়নি আলিশিয়া জিকের। এক ছেলে ও এক মেয়ে দুজনের শরীরেই জন্মগত ত্রুটি রয়েছে। প্রথম সন্তান মেয়ে, যার প্রি-অ্যাক্সিয়াল পলিড্যাক্টিলি সিনড্রোম রয়েছে। এক হাতে মাত্র দুটি আঙ্গুল নিয়ে জন্ম নেয় প্রথম সন্তানটি। আর  এই দম্পতির দ্বিতীয় সন্তান ছেলে, জন্ম থেকেই তার তালুতে চিড় রয়েছে। কোনো দেবদূত বা ঈশ্বর বা যীশু যাই বলেননা কেন, দেখে মনে হচ্ছে এটা আমার প্রতি আশীর্বাদ বলেন মিস্টার স্মিথ। আমি যখন আল্ট্রাসাউন্ড পরীক্ষার সময় এই ছবিটি দেখলাম, চোখে পানি চলে এসেছিল। আমি বাকরুদ্ধ হয়ে পড়েছিলাম। প্রথমে বিশ্বাস হচ্ছিল না। বুধবার আলিশিয়া জিক একটি সুস্থ মেয়ে শিশুর জ্ন্ম দিয়েছেন, তার নাম দেয়া হয়েছে ব্রিয়েলা। মা ও শিশু দুজনেই সুস্থ আছে।" ,
             "যুক্তরাষ্ট্রের এক দম্পতি বলছেন, গর্ভের শিশুর আল্ট্রাসাউন্ড পরীক্ষার সময় তারা 'যীশু খ্রিস্ট'কে দেখতে পেয়েছেন।")
        ],
        "gujarati": [
            ("રાજવી પરિવારોના વીજળી અને પાણીના બિલ સરકારી તિજોરીમાંથી ન ભરવાના નિર્ણયનો આ રાજકુમારો વિરોધ કરી રહ્યા હતા. આ રાજકુમારોના નામ હજુ જાહેર નથી કરાયા.  સાઉદી અરબ સરકારે નિર્ણય કર્યો છે કે રાજવી પરિવાર સાથે જોડાયેલા લોકો પોતાના બિલની ચૂકવણી જાતે જ કરશે. સરકારે જાહેર ખર્ચોમાં કાપ મૂક્યો છે. સરકારે કેટલાય પ્રકારની સબસિડી પણ બંધ કરી છે. સાઉદી અરબે નવા વર્ષથી સ્થાનિક બજારમાં પેટ્રોલ ઉત્પાદનોના ભાવ ડબલ કરી દીધા હતા.  રાજકુમારોના પ્રદર્શનની ખબર સૌપ્રથમ એક સાઉદી વેબસાઇટે પ્રકાશિત કરી હતી. સમાચાર એજન્સી રોઇટર્સ પ્રમાણે સાઉદી અરબના સરકારી વકીલે નિવેદન જાહેર કરી રાજકુમારોની ધરપકડની પુષ્ટિ કરી છે. આ નિવેદનમાં એ પણ કહેવામાં આવ્યું છે કે દેશમાં કાયદાથી ઉપર કોઈ નથી. આ રાજકુમારોને એક અતિ સુરક્ષિત જેલમાં રાખવામાં આવ્યા છે.   ગયા વર્ષે નવેમ્બરમાં ભ્રષ્ટાચાર વિરુદ્ધ લેવાયેલા પગલાંને લઈ કેટલાય રાજકુમારો અને મંત્રીઓની ધરપકડ કરાઈ હતી. સાઉદી અરબના રાજવી પરિવારમાં હજારો લોકો છે. પરંતુ ધન અને સામાજિક આબરૂની દૃષ્ટિએ બધા અલગઅલગ સ્થાને છે.",
             "સાઉદી અરબના એ 11 રાજકુમારની ધરપકડ કરવામાં આવી છે જેમણા રાજમહેલની બહાર પ્રદર્શન કર્યું હતું.")
            ,
            ("સુકમા અને બીજાપુર જિલ્લાની સરહદ પાસે અથડામણ થઈ હોવાના સમાચાર છે.વળી આ અથડામણમાં કેટલાક નક્સલવાદીઓ પણ માર્યાં ગયાના અહેવાલ છે. તાજા અહેવાલો અનુસાર 9 નક્સલી માર્યાં ગયા છે. ન્યૂઝ એજન્સી પીટીઆઈ અનુસાર સુરક્ષા દળો એક નક્સલવિરોધી ઑપરેશન હાથ ધરી રહ્યા હતા ત્યારે અથડામણ થઈ હતી. તેમાં સીઆરપીએફના કોબ્રા કમાન્ડો પણ સામેલ હતા. મુખ્ય મંત્રી ભૂપેશ બઘેલે જવાનોની સારવાર મામલે મહત્વપૂર્ણ નિર્દેશો જારી કર્યા છે. ઘાયલ જવાનોને સારવાર માટે હૉસ્પિટલ ખસેડવામાં આવ્યા છે. કેટલાકને ઍરલિફ્ટ કરી રાયપુર હૉસ્પિટલમાં લઈ જવાયા છે.",
             "છત્તીસગઢમાં નક્સલવાદી જૂથ અને સુરક્ષા દળો વચ્ચે અથડામણમાં એકંદરે 5 જવાનોના મોત થયા છે અને 12થી વધુ ઘાયલ થયા છે.")
        ]
    }

    class SimpleDataset(Dataset):
        def __init__(self, data):
            self.data = data

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            return self.data[idx]

    dataset = SimpleDataset(dummy_data[language])
    return DataLoader(dataset, batch_size=1)



In [ ]:
from transformers import AutoTokenizer
import numpy as np

class GemmaFLClient(fl.client.NumPyClient):
    def __init__(self, model, dataloader):
        self.model = model
        self.dataloader = dataloader
        self.tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
        #self.model.train()

    def get_parameters(self, config):
        return [val.detach().cpu().numpy().astype(np.float32) for val in self.model.parameters()]

    def set_parameters(self, parameters):
        for p, new_val in zip(self.model.parameters(), parameters):
            p.data = torch.tensor(new_val).to(p.device)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=5e-5)

        for input_text, target_text in self.dataloader:
            enc = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
            tgt = self.tokenizer(target_text, return_tensors="pt", padding=True, truncation=True)
            # Move tensors to model device
            inputs = {k: v.to(self.model.device) for k, v in enc.items()}
            targets = tgt["input_ids"].to(self.model.device)

            outputs = self.model(**inputs, labels=targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        return self.get_parameters(config), len(self.dataloader.dataset), {}


    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        total_loss = 0

        with torch.no_grad():
            for input_text, target_text in self.dataloader:
                enc = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
                tgt = self.tokenizer(target_text, return_tensors="pt", padding=True, truncation=True)

                inputs = {k: v.to(self.model.device) for k, v in enc.items()}
                targets = tgt["input_ids"].to(self.model.device)

                outputs = self.model(**inputs, labels=targets)
                total_loss += outputs.loss.item()

        return total_loss / len(self.dataloader), len(self.dataloader.dataset), {}

In [ ]:
def client_fn(cid: str):
    lang_map = {"0": "hindi", "1": "marathi", "2": "bengali", "3": "gujarati"}
    lang = lang_map[cid]
    model = load_model()
    data_loader = load_dataset(lang)
    return GemmaFLClient(model, data_loader)


In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    min_fit_clients=4,
    min_available_clients=4
)

In [ ]:
client_resources = {
    "num_cpus": 0.5,   # or even 0.2 if you're desperate
    "num_gpus": 0.0,   # Avoid GPU usage unless really needed
}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=2,  # try 2 instead of 4
    client_resources=client_resources,
    strategy=strategy,
)
